In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets, metrics, model_selection, linear_model

## Многослойный перцептрон

### Функции активации и функция ошибки

In [2]:
def sigmoid(z):
    """Логистическая функция активации

        Parameters
        ----------
        z: array_like, float
            аргумент функции

        Returns
        -------
        f :
            Результат вычисления функции, принимает значения в промежутке [0, 1]
        """
    return 1.0/(1.0+np.exp(-z))


def sigmoid_derivative(f):
    """Производная логистической функции активации

        Parameters
        ----------
        f : array_like, float
            Результат вычисления функции активации

        Returns
        -------
        f' :
            Результат вычисления производной
        """
    return f * (1 - f)


def tanh(z):
    """Функция активации - гиперболический тангенс

        Parameters
        ----------
        z: array_like, float
            аргумент функции

        Returns
        -------
        f :
            Результат вычисления функции, принимает значения в промежутке [-1, 1]
        """
    return np.tanh(z)


def tanh_derivative(f):
    """Производная гиперболического тангенса

        Parameters
        ----------
        f : array_like, float
            Результат вычисления функции активации

        Returns
        -------
        df : array_like, float
            Результат вычисления производной
        """
    return 1 - np.square(f)


def relu(z):
    """Функция активации - линейный выпрямитель

        Parameters
        ----------
        z: array_like, float
            аргумент функции

        Returns
        -------
        f :
            Результат вычисления функции, принимает значения в промежутке [0, +inf)
        """
    return np.maximum(z, 0)


def relu_derivative(f):
    """Производная линейного выпрямителя

        Parameters
        ----------
        f : array_like, float
            Результат вычисления функции активации

        Returns
        -------
        df : array_like, float
            Результат вычисления производной
        """
    der = np.where(f < 0, 0, 1)
    return der



ACTIVATIONS = {'sigmoid': sigmoid,
               'tanh': tanh,
               'relu': relu}
DERIVATIVES = {'sigmoid': sigmoid_derivative,
               'tanh': tanh_derivative,
               'relu': relu_derivative}

def mean_squared_error(y_pred, y_true):
    """Средняя квадратичная ошибка

        Parameters
        ----------
        y_pred : array_like
            Предсказанные значения ключевого атрибута
        y_true : array_like
            Истинные значения ключевого атрибута

        Returns
        -------
        mse : float
            Средняя квадратичная ошибка
        """
    return 0.5 * (y_pred - y_true) ** 2


def mse_derivative(y_pred, y_true):
    """Производная средней квадратичной ошибки

        Parameters
        ----------
        y_pred : array_like
            Предсказанные значения ключевого атрибута
        y_true : array_like
            Истинные значения ключевого атрибута

        Returns
        -------
        d mse : float
            Производная средней квадратичной ошибки
        """
    return y_pred - y_true

### Реализация многослойного перцептрона

In [3]:
class MultilayerPerceptron:
    """Многослойный перцептрон

        Parameters
        ----------
        layers : array_like
            Список размеров слоёв, где каждый элемент - размер соответствующего слоя.
            Первый слой должен иметь размер, равный количеству атрибутов объектов выборки.
        activation_functions : array_like
            Список функций активации для каждого слоя.
        learning_rate : float
            Скорость обучения, коэффициент, на который на каждом шаге умножается значение градиента.
        """
    def __init__(self, layers, activation_functions, learning_rate = 0.01):
        self.layers_count = len(layers)
        self.activation_functions = activation_functions
        self.layers = layers
        self.weights = [2 * np.random.random((x + 1, y)) - 1 for x, y in zip(layers[:-1], layers[1:])]
        self.learning_rate = learning_rate

    def feedforward(self, x):
        """Прямое распространение ошибки\n
        Для каждого слоя подсчитывается значение функции активации от взвешенных значений предыдущего слоя.\n
        sigma(z), где z = w*x так как для каждого слоя добавляется столбец единиц.


        Parameters
        ----------
        x : array_like
            Объекты обучающей выборки.

        Returns
        -------
        activations : array_like
            Значения функции активации для каждого слоя.
        """
        activations = [np.hstack((np.ones((x.shape[0], 1)), x))] # добавляем столбец единиц для свободного коэффициента
        for i in range(self.layers_count - 2):
            z = np.dot(activations[i], self.weights[i])
            activation = ACTIVATIONS[self.activation_functions[i]](z)
            activations.append(np.hstack((np.ones((activation.shape[0], 1)), activation)))  # добавляем столбец единиц для свободного коэффициента
        activations.append(np.dot(activations[-1], self.weights[-1]))
        return activations

    def backpropagation(self, activations, y):
        """Обратное распространение ошибки\n
        Для изменения весов нейронов внешнего слоя используется значение производной от средней квадратичной ошибки.\n
        Для изменения весов каждого скрытого слоя используются значения ошибки подсчитанные для предыдущего слоя.\n
        Ошибка скрытого слоя равна произведению производной от функции активации на ошибки предыдущего слоя умноженные на веса данного слоя.


        Parameters
        ----------
        activations : array_like
            Изменения весов.
        y : array_like
            Истинные значения ключевого атрибута.

        Returns
        -------
        weight_changes : array_like
            Изменения весов для каждого слоя.
        """
        error = mse_derivative(activations[-1], y)
        weight_changes = [np.average(activations[-2][:, :, np.newaxis] * error[:, np.newaxis, :], axis=0)]
        for i in range(2, self.layers_count):
            error =  DERIVATIVES[self.activation_functions[-i]](activations[-i][:, 1:]) \
                * np.dot(error, self.weights[-i + 1].T[:, 1:])
            delta = activations[-i - 1][:, :, np.newaxis] * error[:, np.newaxis, :]
            weight_changes.append(np.average(delta, axis=0))
        weight_changes.reverse()
        return weight_changes

    def update_weights(self, weight_changes):
        """Обновляет веса в соответствии со значениями градиента

        Parameters
        ----------
        weight_changes : array_like
            Изменения весов.
        """
        for i in range(len(weight_changes) - 1):
            self.weights[i] += - self.learning_rate * weight_changes[i]
        self.weights[-1] += - self.learning_rate * weight_changes[-1]

    def stochastic_gradient_step(self, x, y):
        """Шаг градиентного спуска\n
        На каждом шаге выполняются три действия:
            1) прямое распространение ошибки;
            2) обратное распространение ошибки;
            3) изменения весов.

        Parameters
        ----------
        x : array_like
            Объекты обучающей выборки.
        y : array_like
            Истинные значения ключевого атрибута.
        """
        activations = self.feedforward(x)
        weight_changes = self.backpropagation(activations, y)
        self.update_weights(weight_changes)
        return weight_changes

    def fit(self, x, y, min_weight_dist = 1e-8, max_iter=1e4, batch_size = 5):
        """Запускает процесс обучения сети с помощью алгоритмов
        стохастического градиента спуска и обратного распространения ошибки.

        Parameters
        ----------
        x : array_like
            Объекты обучающей выборки.
        y : array_like
            Истинные значения ключевого атрибута.
        min_weight_dist : float
            Минимальное изменение весов.
        max_iter : float
            Максимальное число итераций градиентного спуска.
        batch_size: int
            Количество объектов в подвыборке
        """
        weight_dist = np.array((np.inf, np.inf))
        iter_num = 0
        while weight_dist.any() > min_weight_dist and iter_num < max_iter:
            random_ind = np.random.randint(x.shape[0])
            w_dist = self.stochastic_gradient_step(x[random_ind:random_ind+batch_size], y[random_ind:random_ind+batch_size])
            weight_dist = []
            for dist in w_dist:
                weight_dist.append(np.linalg.norm(dist))
            weight_dist = np.array(weight_dist)
            iter_num += 1


    def predict(self, x):
        """Предсказывает значения ключевого атрибута на объектах выборки.

        Parameters
        ----------
        x : array_like
            Объекты выборки.

        Returns
        -------
        y : array_like
            Предсказанные значения ключевого атрибута.
        """
        activations = self.feedforward(x)
        return activations[-1]

### Проверка работы реализации многослойного перцептрона на наборе данных "Titanic"

In [4]:
# Импортируем датасет
titanic_df = pd.read_csv("../data/titanic.csv")
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerID  1313 non-null   int64  
 1   Name         1313 non-null   object 
 2   PClass       1313 non-null   object 
 3   Age          756 non-null    float64
 4   Sex          1313 non-null   object 
 5   Survived     1313 non-null   int64  
 6   SexCode      1313 non-null   int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 71.9+ KB


In [5]:
# Проводим предобработку данных
# Заполняем пропуски средними значениями
titanic_df = titanic_df.fillna(titanic_df.mean())

# Перекодируем категориальный атрибут
class_mapping = {'1st': 1,
                 '2nd': 2,
                 '3rd': 3,
                 '*': 4 }

titanic_df['PClass'] = titanic_df['PClass'].map(class_mapping)
titanic_df['PClass'] = titanic_df['PClass'].astype('int64', copy=False)

# Отбрасываем ненужный признак, так как у нас есть признак "SexCode"
titanic_df = titanic_df.drop(columns='Sex')

In [6]:
# Разделяем выборку, а также отбрасываем бесполезные атрибуты 'Name' и 'PassengerID'
X_titanic = titanic_df.drop(columns=['Survived', 'Name', 'PassengerID'])
y_titanic = titanic_df['Survived']

In [7]:
# Разделяем выборку на обучающую и тестирующую
X_titanic = np.array(X_titanic)
y_titanic = np.array(y_titanic).reshape(y_titanic.shape[0], 1)

X_titanic_train, X_titanic_test, y_titanic_train, y_titanic_test = model_selection.train_test_split(X_titanic, y_titanic, train_size=0.75)

In [47]:
# Инициализируем многослойный перцептрон и логистическую регрессию
titanic_lc = linear_model.LogisticRegression()
titanic_mp = MultilayerPerceptron([3, 5, 4, 3, 1], ['sigmoid', 'sigmoid', 'sigmoid', 'sigmoid', 'sigmoid'], 0.05)

In [48]:
# Обучаем модели и получаем результаты
titanic_mp.fit(X_titanic_train, y_titanic_train)
titanic_mp_pred = titanic_mp.predict(X_titanic_test)

In [10]:
titanic_lc_train = y_titanic_train.ravel()
titanic_lc.fit(X_titanic_train, titanic_lc_train)
titanic_lc_pred = titanic_lc.predict(X_titanic_test)

C:\Users\Andrew\AppData\Roaming\Python\Python38\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [49]:
# Переводим предсказания вероятностей в предсказания классов
titanic_classes = []
for pred in titanic_mp_pred:
    if pred >= 0.5:
        titanic_classes.append(1)
    else:
        titanic_classes.append(0)

In [50]:
# Выводим результаты работы моделей
print("Multilayer Perceptron:")
print(metrics.confusion_matrix(y_titanic_test, titanic_classes))
print(metrics.classification_report(y_titanic_test, titanic_classes))

print("\nLogistic Regression:")
print(metrics.confusion_matrix(y_titanic_test, titanic_lc_pred))
print(metrics.classification_report(y_titanic_test, titanic_lc_pred))

Multilayer Perceptron:
[[204  17]
 [ 56  52]]
              precision    recall  f1-score   support

           0       0.78      0.92      0.85       221
           1       0.75      0.48      0.59       108

    accuracy                           0.78       329
   macro avg       0.77      0.70      0.72       329
weighted avg       0.77      0.78      0.76       329


Logistic Regression:
[[207  14]
 [ 44  64]]
              precision    recall  f1-score   support

           0       0.82      0.94      0.88       221
           1       0.82      0.59      0.69       108

    accuracy                           0.82       329
   macro avg       0.82      0.76      0.78       329
weighted avg       0.82      0.82      0.82       329



Видно, что сеть с двумя скрытыми слоями отработала чуть хуже линейной без подбора параметров

### Проверка работы реализации многослойного перцептрона на наборе данных "Iris"

In [13]:
# Загружаем данные и разделяем их на обучающую и тестовую выборки

iris = datasets.load_iris()
X_iris = iris.data
y_iris = iris.target
y_iris = y_iris.reshape(y_iris.shape[0], 1)

X_iris_train, X_iris_test, y_iris_train, y_iris_test = model_selection.train_test_split(X_iris, y_iris, train_size=0.8)

In [19]:
# Инициализируем модели

iris_lr = linear_model.LogisticRegression()
iris_mp = MultilayerPerceptron([4, 10, 10, 1], ['sigmoid', 'sigmoid', 'sigmoid', 'sigmoid'], 0.1)

In [20]:
# Обучаем модели
iris_mp.fit(X_iris_train, y_iris_train)
iris_mp_pred = iris_mp.predict(X_iris_test)
iris_classes = []
for pred in iris_mp_pred:
    if pred < 0.5:
        iris_classes.append(0)
    elif pred <= 1.5:
        iris_classes.append(1)
    elif pred <= 2.5:
        iris_classes.append(2)

In [23]:
y_iris_lr = y_iris_train.ravel()
iris_lr.fit(X_iris_train, y_iris_lr)
iris_lr_pred = iris_lr.predict(X_iris_test)

In [24]:
# Выводим результаты работы моделей
print("Multilayer Perceptron:")
print(metrics.confusion_matrix(y_iris_test, iris_classes))
print(metrics.classification_report(y_iris_test, iris_classes))

print("\nLogistic Regression:")
print(metrics.confusion_matrix(y_iris_test, iris_lr_pred))
print(metrics.classification_report(y_iris_test, iris_lr_pred))

Multilayer Perceptron:
[[10  0  0]
 [ 0  8  1]
 [ 0  0 11]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.89      0.94         9
           2       0.92      1.00      0.96        11

    accuracy                           0.97        30
   macro avg       0.97      0.96      0.97        30
weighted avg       0.97      0.97      0.97        30


Linear Regression:
[[10  0  0]
 [ 0  8  1]
 [ 0  1 10]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.89      0.89      0.89         9
           2       0.91      0.91      0.91        11

    accuracy                           0.93        30
   macro avg       0.93      0.93      0.93        30
weighted avg       0.93      0.93      0.93        30



Видно, что на данном наборе данных четырёхслойная сеть отработала на уровне с линейной

### Проверка работы реализации многослойного перцептрона на наборе данных "Auto mpg"

In [51]:
# Импортируем набор данных, разделяем на обучающую и тестирующую выборки
mpg_df = pd.read_csv("../data/auto_mpg_preprocessed.csv")
mpg_df = mpg_df.drop(columns=["Unnamed: 0", 'car name'])

X_mpg = mpg_df.drop(columns='mpg')
y_mpg = mpg_df['mpg']

y_mpg = np.array(y_mpg)
y_mpg = y_mpg.reshape(y_mpg.shape[0], 1)

X_mpg_train, X_mpg_test, y_mpg_train, y_mpg_test = model_selection.train_test_split(X_mpg, y_mpg, train_size=0.75)

In [52]:
# Обучаем модели на масштабированных данных
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
X_mpg_train = scaler.fit_transform(X_mpg_train, y_mpg_train)
X_mpg_test = scaler.transform(X_mpg_test)

mpg_mp = MultilayerPerceptron([7, 15, 15, 1], ['sigmoid', 'sigmoid', 'sigmoid', 'sigmoid'])
mpg_mp.fit(X_mpg_train, y_mpg_train)
mpg_mp_predict = mpg_mp.predict(X_mpg_test)

In [53]:
mpg_lr = linear_model.LinearRegression()
mpg_lr.fit(X_mpg_train, y_mpg_train.ravel())
mpg_lr_predict = mpg_lr.predict(X_mpg_test)

In [56]:
# Выводим результаты

print("Multilayer Perceptron:")
print("MSE: ", metrics.mean_squared_error(y_mpg_test, mpg_mp_predict))
print("MAE: ", metrics.mean_absolute_error(y_mpg_test, mpg_mp_predict))
print("R2: ", metrics.r2_score(y_mpg_test, mpg_mp_predict))

print("\nLinear Regression:")
print("MSE: ", metrics.mean_squared_error(y_mpg_test, mpg_lr_predict))
print("MAE: ", metrics.mean_absolute_error(y_mpg_test, mpg_lr_predict))
print("R2: ", metrics.r2_score(y_mpg_test, mpg_lr_predict))

Multilayer Perceptron:
MSE:  4.8514021593132055
MAE:  1.6903907213857643
R2:  0.9008892713715522

Linear Regression:
MSE:  8.252112074971302
MAE:  2.3470766201678424
R2:  0.8314151633659244


Видно, что на данном наборе данных сеть отработала лучше линейной регрессии

### Проверка работы реализации многослойного перцептрона на наборе данных "Wine"

In [57]:
# Импортируем набор данных, разделяем на обучающую и тестирующую выборки
wine_df = pd.read_csv("../data/wine_preprocessed.csv")
wine_df = wine_df.drop(columns=["Unnamed: 0"])

X_wine = wine_df.drop(columns='Cultivar')
y_wine = wine_df['Cultivar']

y_wine = np.array(y_wine)
y_wine = y_wine.reshape(y_wine.shape[0], 1)

X_wine_train, X_wine_test, y_wine_train, y_wine_test = model_selection.train_test_split(X_wine, y_wine, train_size=0.75)

In [58]:
# Обучаем модели на масштабированных данных
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
X_wine_train = scaler.fit_transform(X_wine_train, y_wine_train)
X_wine_test = scaler.transform(X_wine_test)

wine_mp = MultilayerPerceptron([13, 15, 15, 1], ['sigmoid', 'sigmoid', 'sigmoid', 'sigmoid'])
wine_mp.fit(X_wine_train, y_wine_train)
wine_mp_predict = wine_mp.predict(X_wine_test)

In [60]:
wine_classes = []
for pred in wine_mp_predict:
    if pred < 0.5:
        wine_classes.append(0)
    elif pred <= 1.5:
        wine_classes.append(1)
    elif pred <= 2.5:
        wine_classes.append(2)
    else:
        wine_classes.append(3)


In [61]:
wine_lc = linear_model.LogisticRegression()
wine_lc.fit(X_wine_train, y_wine_train.ravel())
wine_lc_predict = wine_lc.predict(X_wine_test)

In [62]:
# Выводим результаты
print("Multilayer Perceptron:")
print(metrics.confusion_matrix(y_wine_test, wine_classes))
print(metrics.classification_report(y_wine_test, wine_classes))

print("\nLogistic Regression:")
print(metrics.confusion_matrix(y_wine_test, wine_lc_predict))
print(metrics.classification_report(y_wine_test, wine_lc_predict))


Multilayer Perceptron:
[[16  0  0]
 [ 1 15  1]
 [ 0  0 12]]
              precision    recall  f1-score   support

           1       0.94      1.00      0.97        16
           2       1.00      0.88      0.94        17
           3       0.92      1.00      0.96        12

    accuracy                           0.96        45
   macro avg       0.95      0.96      0.96        45
weighted avg       0.96      0.96      0.95        45


Logistic Regression:
[[16  0  0]
 [ 0 17  0]
 [ 0  0 12]]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        16
           2       1.00      1.00      1.00        17
           3       1.00      1.00      1.00        12

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



Видно, что на данном наборе данных сеть отработала примерно на одном уровне с логистической регрессией